# Học biểu diễn dữ liệu graph sử dụng GraphSage

## Dataset

### Protein-Protein interaction

![image.png](attachment:image.png)

Tương tác protein là hiện tượng vật lý mà hai hay nhiều protein bám vào nhau, là trung tâm của mọi quá trình sinh học diễn ra trong tế bào.
Các proteins được biểu diễn bằng các nodes, liên kết bởi các edges vô hướng.

Tương tác protein-protein (PPI) là điều cần thiết cho hầu hết mọi quá trình trong tế bào, vì vậy hiểu PPI là rất quan trọng để hiểu sinh lý tế bào ở trạng thái bình thường và bệnh. Nó cũng rất cần thiết trong việc phát triển thuốc, vì thuốc có thể ảnh hưởng đến PPI.

#### Định dạng dữ liệu

* toy-ppi-G.json -- A networkx-specified json file describing the input graph. Nodes have 'val' and 'test' attributes specifying if they are a part of the validation and test sets, respectively.
* toy-ppi-id_map.json -- A json-stored dictionary mapping the graph node ids to consecutive integers.
* toy-ppi-class_map.json -- A json-stored dictionary mapping the graph node ids to classes.
* toy-ppi-feats.npy [optional] --- A numpy-stored array of node features; ordering given by id_map.json. Can be omitted and only identity features will be used.
* toy-ppi-walks.txt [optional] --- A text file specifying random walk co-occurrences (one pair per line) (*only for unsupervised version of graphsage)

## Thử nghiệm trên dữ liệu PPI

### Phân loại Protein chỉ sử dụng Features

In [2]:
!unrar x "toy-ppi_data.rar" "/content/"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from toy-ppi_data.rar

Creating    /content/toy-ppi_data                                     OK
Extracting  /content/toy-ppi_data/toy-ppi-class_map.json                   5%  OK 
Extracting  /content/toy-ppi_data/toy-ppi-feats.npy                        6%  OK 
Extracting  /content/toy-ppi_data/toy-ppi-G.json                          26%  OK 
Extracting  /content/toy-ppi_data/toy-ppi-id_map.json                     26%  OK 
Extracting  /content/toy-ppi_data/toy-ppi-walks.txt                       93% 99%  OK 
All OK


In [1]:
import numpy as np
from networkx.readwrite import json_graph
import json
import os
cwd = os.getcwd()

dataset_dir = '/content/toy-ppi_data'

Tải bộ dữ liệu *toy-ppi* từ thư mục *'example_data'* vào đối tượng Graph của networkx. Phân tách các nodes thành tập huấn luyện và tập kiểm thử.

In [3]:
print("Loading data...")
G = json_graph.node_link_graph(json.load(open(dataset_dir + "/toy-ppi-G.json")))
labels = json.load(open(dataset_dir + "/toy-ppi-class_map.json"))
labels = {int(i):l for i, l in labels.items()}
    
train_ids = [n for n in G.nodes() if not G.nodes[n]['val'] and not G.nodes[n]['test']]
test_ids = [n for n in G.nodes() if G.nodes[n]['test']]
train_labels = np.array([labels[i] for i in train_ids])
if train_labels.ndim == 1:
    train_labels = np.expand_dims(train_labels, 1)
test_labels = np.array([labels[i] for i in test_ids])
print("Loaded data")

Loading data...
Loaded data


Hàm huấn luyện mô hình hồi quy trên dữ liệu

In [4]:
def run_regression(train_embeds, train_labels, test_embeds, test_labels):
    np.random.seed(1)
    from sklearn.linear_model import SGDClassifier
    from sklearn.dummy import DummyClassifier
    from sklearn.metrics import f1_score
    from sklearn.multioutput import MultiOutputClassifier
    dummy = MultiOutputClassifier(DummyClassifier())
    dummy.fit(train_embeds, train_labels)
    log = MultiOutputClassifier(SGDClassifier(loss="log", max_iter=5, tol=-np.infty), n_jobs=10)
    log.fit(train_embeds, train_labels)

    f1 = 0
    for i in range(test_labels.shape[1]):
        print("F1 score", f1_score(test_labels[:,i], log.predict(test_embeds)[:,i], average="micro"))
    for i in range(test_labels.shape[1]):
        print("Random baseline F1 score", f1_score(test_labels[:,i], dummy.predict(test_embeds)[:,i], average="micro"))

Đọc và sử dụng trực tiếp features của proteins cho phân loại 

In [6]:
feats = np.load(dataset_dir + "/toy-ppi-feats.npy")
## Logistic gets thrown off by big counts, so log transform num comments and score
feats[:,0] = np.log(feats[:,0]+1.0)
feats[:,1] = np.log(feats[:,1]-min(np.min(feats[:,1]), -1))
feat_id_map = json.load(open(dataset_dir + "/toy-ppi-id_map.json"))
feat_id_map = {int(id):val for id,val in feat_id_map.items()}
train_feats = feats[[feat_id_map[id] for id in train_ids]] 
test_feats = feats[[feat_id_map[id] for id in test_ids]] 
print("Running regression..")
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_feats)
train_feats = scaler.transform(train_feats)
test_feats = scaler.transform(test_feats)
run_regression(train_feats, train_labels, test_feats, test_labels)

Running regression..


/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


F1 score 0.5084007467330429
F1 score 0.7165525824517733
F1 score 0.718108276291226
F1 score 0.6670815183571873
F1 score 0.8254511512134411
F1 score 0.7685127566894835
F1 score 0.6922837585563161
F1 score 0.6148102053515868
F1 score 0.7825140012445551
F1 score 0.6157436216552582
F1 score 0.7781580584940884
F1 score 0.6692594897324207
F1 score 0.4598630989421282
F1 score 0.7420659614187929
F1 score 0.7654013690105788
F1 score 0.5728064716863721
F1 score 0.8459863098942129
F1 score 0.7918481642812694
F1 score 0.6698817672682016
F1 score 0.5581829495955196
F1 score 0.7778469197261979
F1 score 0.4915992532669571
F1 score 0.6966397013067829
F1 score 0.47075295581829496
F1 score 0.7756689483509647
F1 score 0.5612943372744243
F1 score 0.4710640945861854
F1 score 0.6160547604231488
F1 score 0.6210329807093964
F1 score 0.7501555693839452
F1 score 0.6042314872433105
F1 score 0.7868699439950217
F1 score 0.8397635345364033
F1 score 0.7594897324206596
F1 score 0.7691350342252644
F1 score 0.616677037

### Phân loại Proteins sử dụng protein features và topology information

Sử dụng kỹ thuật GraphSage để kết hợp đặc trưng về topology cho bài toán phân loại Proteins. Trước tiên, clone GraphSage từ github:

In [7]:
!git clone https://github.com/williamleif/GraphSAGE.git

Cloning into 'GraphSAGE'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 262 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (262/262), 6.43 MiB | 12.91 MiB/s, done.
Resolving deltas: 100% (158/158), done.


Chạy module unsupervised_train trên tập dữ liệu toy-ppi, với mean-based aggregator.
Học viên có thể thử thay đổi aggregator: graphsage_seq; graphsage_maxpool; graphsage_meanpool; gcn; n2v

In [8]:
%cd GraphSAGE/

/content/GraphSAGE


In [20]:
%tensorflow_version 1.8

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.8`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [10]:
!pip install -r requirements.txt

     |████████████████████████████████| 92kB 5.7MB/s 
     |████████████████████████████████| 9.0MB 1.3MB/s 
     |████████████████████████████████| 890kB 50.4MB/s 
     |████████████████████████████████| 81kB 12.0MB/s 
     |████████████████████████████████| 61kB 10.2MB/s 
     |████████████████████████████████| 1.3MB 49.2MB/s 
     |████████████████████████████████| 12.2MB 15.9MB/s 
     |████████████████████████████████| 102kB 14.4MB/s 
     |████████████████████████████████| 7.1MB 49.4MB/s 
     |████████████████████████████████| 31.2MB 110kB/s 
     |████████████████████████████████| 3.1MB 45.9MB/s 
     |████████████████████████████████| 49.1MB 134kB/s 
     |████████████████████████████████| 327kB 46.3MB/s 
  Created wheel for absl-py: filename=absl_py-0.2.2-cp36-none-any.whl size=98947 sha256=11e270c7e8624dfa508508aea6866ce114460a4ab6f68934d3afba710d8d1cc9
  Stored in directory: /root/.cache/pip/wheels/a0/f8/e9/1933dbb3447ea6ef57062fd5461cb118deb8c2ed074e8344bf
  Created wheel 

In [11]:
!python -m graphsage.unsupervised_train --train_prefix ./example_data/toy-ppi --model graphsage_mean --max_total_steps 1000 --validate_iter 10

Loading training data..
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
Loaded data.. now preprocessing..
Done loading training data..
Unexpected missing: 0
9716 train nodes
5039 test nodes
2020-12-11 04:51:26.541420: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Epoch: 0001
Iter: 0000 train_loss= 18.78067 train_mrr= 0.23649 train_mrr_ema= 0.23649 val_loss= 19.08406 val_mrr= 0.18671 val_mrr_ema= 0.18671 time= 0.83781
Iter: 0050 train_loss= 18.47410 train_mrr= 0.17529 train_mrr_ema= 0.21827 val_loss= 18.57477 val_mrr= 0.22494 val_mrr_ema= 0.18911 time= 0.30174
Iter: 0100 train_loss= 18.42166 train_mrr= 0.17687 train_mrr_ema= 0.20626 val_loss= 18.20559 val_mrr= 0.20144 val_mrr_ema= 0.19924 time= 0.29699
Iter: 0150 train_loss= 17.77015 train_mrr= 0.19358 train_mrr_ema= 0.19745 val_loss= 18.07098 val_mrr= 0.21800 val_mrr_ema= 0.20326 time= 0.29506
Iter: 0200 

Kết quả embed được lưu xuống tệp *'val.npy'*, gồm các vector *d* chiều biểu diễn các nodes của ppi trong không gian vector.

In [12]:
data_dir = 'unsup-example_data/graphsage_mean_small_0.000010'

embeds = np.load(data_dir + "/val.npy")
print(embeds)
print(embeds.shape)

[[ 0.08493663 -0.09270122  0.08393622 ...  0.01101307  0.0197683
  -0.02167838]
 [-0.0138827   0.0685497  -0.00072682 ... -0.09227714 -0.00045772
   0.0740908 ]
 [-0.03758346  0.02042661  0.05431975 ...  0.04314912  0.09420752
  -0.07455792]
 ...
 [ 0.04227313 -0.12351843 -0.04573519 ... -0.00327433  0.01051723
  -0.00968124]
 [-0.05660874  0.09731396 -0.09867457 ...  0.02644388  0.13637076
   0.03107577]
 [ 0.01826782 -0.08390719 -0.04281924 ...  0.04707416  0.00856595
   0.07259557]]
(14755, 256)


Thực hiện phân loại proteins sử dụng mô hình hồi quy trên dữ liệu PPI đã embed 

In [14]:
id_map = {}
with open(data_dir + "/val.txt") as fp:
    for i, line in enumerate(fp):
        id_map[int(line.strip())] = i
train_embeds = embeds[[id_map[id] for id in train_ids]] 
test_embeds = embeds[[id_map[id] for id in test_ids]] 

print("Running regression..")
# chạy regression
#### YOUR CODE HERE ####
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_embeds)
train_embeds = scaler.transform(train_embeds)
test_embeds = scaler.transform(test_embeds)
run_regression(train_embeds, train_labels, test_embeds, test_labels)

Running regression..


/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


F1 score 0.5871188550093341
F1 score 0.6922837585563161
F1 score 0.7271313005600498
F1 score 0.6627255756067206
F1 score 0.7650902302426882
F1 score 0.7321095208462974
F1 score 0.6708151835718731
F1 score 0.5952084629744866
F1 score 0.7853142501555694
F1 score 0.5880522713130056
F1 score 0.7212196639701307
F1 score 0.6876166770379589
F1 score 0.574673304293715
F1 score 0.7174859987554449
F1 score 0.7495332918481643
F1 score 0.5672059738643435
F1 score 0.8385189794648413
F1 score 0.746110765401369
F1 score 0.6605476042314873
F1 score 0.5637834474175483
F1 score 0.7433105164903546
F1 score 0.568139390168015
F1 score 0.6907280647168638
F1 score 0.5945861854387057
F1 score 0.7700684505289359
F1 score 0.5644057249533292
F1 score 0.5693839452395768
F1 score 0.6291225886745488
F1 score 0.5628500311138768
F1 score 0.7392657125077784
F1 score 0.6157436216552582
F1 score 0.783758556316117
F1 score 0.8027380211574362
F1 score 0.7498444306160548
F1 score 0.7103298070939638
F1 score 0.6400124455507